In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from hts import HTSRegressor
from hts.hierarchy import HierarchyTree

/home/mark/miniconda3/envs/aim/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Get Prices Table
Process prices table into suitable format

In [2]:
df1 = pd.read_csv('../data/aim_prices.csv', index_col=0)
mask = df1['Adj Close'].isnull()
df1[mask]

,Date,Open,High,Low,Close,Adj Close,Volume,symbol
896,2020-07-20,NaN,NaN,NaN,NaN,NaN,NaN,PAF
1457,2020-07-20,NaN,NaN,NaN,NaN,NaN,NaN,TM17
2368,2020-07-20,NaN,NaN,NaN,NaN,NaN,NaN,IOM
3279,2020-07-20,NaN,NaN,NaN,NaN,NaN,NaN,YNGN
4112,2020-07-20,NaN,NaN,NaN,NaN,NaN,NaN,ESL
...,...,...,...,...,...,...,...,...
83564,2020-07-20,NaN,NaN,NaN,NaN,NaN,NaN,IQE
84475,2020-07-20,NaN,NaN,NaN,NaN,NaN,NaN,FDP
85386,2020-07-20,NaN,NaN,NaN,NaN,NaN,NaN,VCP
85656,2020-07-20,NaN,NaN,NaN,NaN,NaN,NaN,UPR


In [3]:
to_drop = ['Open', 'High', 'Low', 'Close', 'Volume']
df2 = df1.drop(columns=to_drop)
df2.fillna(method='ffill', inplace=True)
df2[mask]

,Date,Adj Close,symbol
896,2020-07-20,22.000000,PAF
1457,2020-07-20,628.000000,TM17
2368,2020-07-20,332.500000,IOM
3279,2020-07-20,667.500000,YNGN
4112,2020-07-20,7.850000,ESL
...,...,...,...
83564,2020-07-20,47.419998,IQE
84475,2020-07-20,2600.000000,FDP
85386,2020-07-20,240.000000,VCP
85656,2020-07-20,1.775000,UPR


In [4]:
df3 = df2.pivot_table(index='Date', values='Adj Close', columns='symbol', aggfunc='first')
df3

symbol,ABC,ABDP,AFM,AFX,AMS,APGN,APH,ASC,ASY,ATYM,...,TRMR,TUNE,UPR,VCP,WAND,WHR,WJG,YNGA,YNGN,YOU
Date,,,,,,,,,,,,,,,,,,,,,
2017-01-03,747.474426,544.871216,NaN,NaN,217.710587,366.378662,42.915035,5081.0,359.247223,123.0,...,175.5,209.348175,NaN,360.0,195.5,NaN,104.183655,1283.661255,919.234985,237.588608
2017-01-04,735.387268,537.453857,NaN,NaN,216.236252,373.805267,44.329815,5038.0,359.247223,125.0,...,178.0,207.880783,NaN,369.0,196.5,NaN,104.183655,1283.180786,930.601013,240.037949
2017-01-05,740.222107,540.915039,NaN,NaN,214.270432,373.805267,44.565609,5124.0,355.094055,137.5,...,175.5,207.880783,NaN,367.5,175.0,NaN,103.631836,1283.180786,937.468140,233.424683
2017-01-06,741.189087,530.037415,NaN,NaN,211.813232,388.658447,44.683510,5256.0,355.094055,142.0,...,175.5,203.967728,NaN,367.5,175.0,NaN,103.300735,1273.568848,937.704895,244.936707
2017-01-09,746.991028,532.510193,NaN,NaN,211.321777,396.085022,45.037201,5315.0,355.094055,151.5,...,180.0,203.967728,NaN,360.0,175.0,NaN,102.859283,1287.505859,935.337097,244.936707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-03,1300.000000,1675.000000,190.0,830.0,194.399994,330.000000,71.300003,3465.0,606.299988,208.0,...,123.0,720.000000,2.055,270.0,538.0,110.5,137.199997,870.000000,692.500000,810.000000
2020-08-04,1259.000000,1675.000000,193.0,850.0,193.800003,330.000000,72.599998,3523.0,606.299988,209.0,...,121.0,710.000000,2.055,281.0,540.0,110.0,136.199997,910.000000,692.500000,815.000000
2020-08-05,1254.000000,1720.000000,194.0,880.0,198.399994,330.000000,69.099998,3899.0,606.299988,208.5,...,126.0,710.000000,2.055,283.0,600.0,110.0,139.399994,980.000000,692.500000,820.000000


In [5]:
df3.isnull().sum()[df3.isnull().sum() > 0]

symbol
AFM     196
AFX      68
ASY       1
CDM     356
ESL      78
GRP     141
JSE     404
KETL    151
KGH     376
LGRS    586
PEBB    742
SUMO    247
TM17    350
UPR     641
WHR     181
dtype: int64

In [6]:
df4 = df3.dropna(axis=1)
df4

symbol,ABC,ABDP,AMS,APGN,APH,ASC,ATYM,BMK,BMN,BOO,...,TFW,TRCS,TRMR,TUNE,VCP,WAND,WJG,YNGA,YNGN,YOU
Date,,,,,,,,,,,,,,,,,,,,,
2017-01-03,747.474426,544.871216,217.710587,366.378662,42.915035,5081.0,123.0,81.362900,1.925,1.370000,...,302.201447,502.611359,175.5,209.348175,360.0,195.5,104.183655,1283.661255,919.234985,237.588608
2017-01-04,735.387268,537.453857,216.236252,373.805267,44.329815,5038.0,125.0,81.362900,1.775,1.340000,...,300.312653,497.659546,178.0,207.880783,369.0,196.5,104.183655,1283.180786,930.601013,240.037949
2017-01-05,740.222107,540.915039,214.270432,373.805267,44.565609,5124.0,137.5,82.860397,1.825,1.385000,...,293.702026,497.659546,175.5,207.880783,367.5,175.0,103.631836,1283.180786,937.468140,233.424683
2017-01-06,741.189087,530.037415,211.813232,388.658447,44.683510,5256.0,142.0,82.860397,1.975,1.407500,...,271.036926,497.659546,175.5,203.967728,367.5,175.0,103.300735,1273.568848,937.704895,244.936707
2017-01-09,746.991028,532.510193,211.321777,396.085022,45.037201,5315.0,151.5,83.858704,2.025,1.435000,...,288.035797,505.087280,180.0,203.967728,360.0,175.0,102.859283,1287.505859,935.337097,244.936707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-03,1300.000000,1675.000000,194.399994,330.000000,71.300003,3465.0,208.0,41.000000,11.875,270.299988,...,297.000000,580.000000,123.0,720.000000,270.0,538.0,137.199997,870.000000,692.500000,810.000000
2020-08-04,1259.000000,1675.000000,193.800003,330.000000,72.599998,3523.0,209.0,40.500000,13.000,265.000000,...,290.000000,580.000000,121.0,710.000000,281.0,540.0,136.199997,910.000000,692.500000,815.000000
2020-08-05,1254.000000,1720.000000,198.399994,330.000000,69.099998,3899.0,208.5,41.000000,13.500,275.000000,...,295.000000,580.000000,126.0,710.000000,283.0,600.0,139.399994,980.000000,692.500000,820.000000


In [7]:
symbols = df4.columns
symbols

Index(['ABC', 'ABDP', 'AMS', 'APGN', 'APH', 'ASC', 'ATYM', 'BMK', 'BMN', 'BOO',
       'BREE', 'BRK', 'BUR', 'BVXP', 'CAM', 'CAML', 'CHRT', 'CLIN', 'CREO',
       'CRW', 'CTH', 'CVSG', 'CWR', 'DATA', 'DOTD', 'DTG', 'EMIS', 'FDEV',
       'FDP', 'FEVR', 'GAMA', 'GBG', 'GHH', 'GROW', 'GTLY', 'HCM', 'HGM',
       'HOTC', 'HUR', 'HZD', 'IDEA', 'IGR', 'IMO', 'IOM', 'IPX', 'IQE', 'ITM',
       'JHD', 'JSG', 'KAPE', 'KWS', 'LTG', 'MAB1', 'MIDW', 'MPE', 'MRL', 'MTW',
       'NFC', 'NICL', 'NUM', 'PAF', 'PMI', 'POLR', 'PRSM', 'PURP', 'RNWH',
       'RQIH', 'RST', 'RWS', 'SCPA', 'SIR', 'SLN', 'SMS', 'SQZ', 'TCM', 'TFW',
       'TRCS', 'TRMR', 'TUNE', 'VCP', 'WAND', 'WJG', 'YNGA', 'YNGN', 'YOU'],
      dtype='object', name='symbol')

# Get Groupings
Get groupings table

In [8]:
dfg = pd.read_csv('../data/aim_info.csv', index_col=0)
dfg

,address1,address2,city,zip,country,phone,fax,website,industry,sector,longBusinessSummary,fullTimeEmployees,companyOfficers,maxAge,symbol
0,The Firs Office Building,"2nd Floor, Office 204 Cnr Cradock and Biermann...",Johannesburg,2196,South Africa,27 11 243 2900,27 86 214 1090,http://www.panafricanresources.com,Gold,Basic Materials,Pan African Resources PLC engages in the explo...,2148.0,[],86400,PAF
1,3 Red Hall Avenue,Paragon Business Park,Wakefield,WF1 2UL,United Kingdom,44 19 2426 7776,NaN,http://www.team17.com,Electronic Gaming & Multimedia,Communication Services,"Team17 Group PLC, together with its subsidiari...",200.0,[],86400,TM17
2,West of Scotland Science Park,Kelvin Campus Lister Pavilion,Glasgow,G20 0SP,United Kingdom,44 14 1931 6400,NaN,http://www.iomart.com,Software—Infrastructure,Technology,iomart Group plc provides managed cloud servic...,405.0,[],86400,IOM
3,Riverside House,26 Osiers Road Wandsworth,London,SW18 1NH,United Kingdom,44 20 8875 7000,44 20 8875 7100,http://www.youngs.co.uk,Restaurants,Consumer Cyclical,"Young & Co.'s Brewery, P.L.C. operates and man...",5145.0,[],86400,YNGN
4,Stretton Green Distribution Park,Langford Way Appleton,Warrington,WA4 4TQ,United Kingdom,44 19 2560 5400,NaN,http://www.eddiestobart.com,Integrated Freight & Logistics,Industrials,Eddie Stobart Logistics plc provides logistics...,NaN,[],86400,ESL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,Pascal Close,St. Mellons,Cardiff,CF3 0LW,United Kingdom,44 29 2083 9400,44 29 2079 4592,http://www.iqep.com,Semiconductor Equipment & Materials,Technology,"IQE plc develops, manufactures, and sells adva...",663.0,[],86400,IQE
93,3 Canal Quay,NaN,Newry,BT35 6BP,United Kingdom,44 28 3025 2242,44 28 3025 2060,http://www.firstderivatives.com,Information Technology Services,Technology,First Derivatives plc provides software and co...,2400.0,[],86400,FDP
94,Worcester Road,NaN,Kidderminster,DY10 1JR,United Kingdom,44 15 6274 9300,44 15 6274 9649,http://www.victoriaplc.com,"Furnishings, Fixtures & Appliances",Consumer Cyclical,"Victoria PLC designs, manufactures, and distri...",3400.0,[],86400,VCP
95,4045 Kingswood Road,Citywest Business Park,Dublin,D24 V06K,Ireland,353 1 428 7777,353 1 428 7776,http://www.uniphar.ie,Medical Care Facilities,Healthcare,Uniphar plc operates as a diversified healthca...,2200.0,[],86400,UPR


In [9]:
to_drop = dfg[dfg['sector'].isnull()].index
dfg2 = dfg.drop(to_drop)

In [10]:
to_drop = dfg2[~dfg2['symbol'].isin(symbols)].index
dfg3 = dfg2.drop(to_drop)
dfg3

,address1,address2,city,zip,country,phone,fax,website,industry,sector,longBusinessSummary,fullTimeEmployees,companyOfficers,maxAge,symbol
0,The Firs Office Building,"2nd Floor, Office 204 Cnr Cradock and Biermann...",Johannesburg,2196,South Africa,27 11 243 2900,27 86 214 1090,http://www.panafricanresources.com,Gold,Basic Materials,Pan African Resources PLC engages in the explo...,2148.0,[],86400,PAF
2,West of Scotland Science Park,Kelvin Campus Lister Pavilion,Glasgow,G20 0SP,United Kingdom,44 14 1931 6400,NaN,http://www.iomart.com,Software—Infrastructure,Technology,iomart Group plc provides managed cloud servic...,405.0,[],86400,IOM
3,Riverside House,26 Osiers Road Wandsworth,London,SW18 1NH,United Kingdom,44 20 8875 7000,44 20 8875 7100,http://www.youngs.co.uk,Restaurants,Consumer Cyclical,"Young & Co.'s Brewery, P.L.C. operates and man...",5145.0,[],86400,YNGN
5,Greater London House,Hampstead Road,London,NW1 7FB,United Kingdom,44 20 7756 1000,NaN,http://www.asosplc.com,Internet Retail,Consumer Cyclical,ASOS Plc operates as an online fashion retaile...,4000.0,[],86400,ASC
6,Vinces Road,NaN,Diss,IP22 4YT,United Kingdom,44 1379 649200,NaN,http://www.midwichgroupplc.com,Electronics & Computer Distribution,Technology,Midwich Group Plc distributes audio visual (AV...,1000.0,[],86400,MIDW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,Middleton Drive,NaN,Bradford-on-Avon,BA15 1GB,United Kingdom,44 12 2586 0200,44 12 2586 0201,http://www.abdplc.com,Auto Parts,Consumer Cyclical,"AB Dynamics plc, through its subsidiaries, des...",264.0,[],86400,ABDP
92,Pascal Close,St. Mellons,Cardiff,CF3 0LW,United Kingdom,44 29 2083 9400,44 29 2079 4592,http://www.iqep.com,Semiconductor Equipment & Materials,Technology,"IQE plc develops, manufactures, and sells adva...",663.0,[],86400,IQE
93,3 Canal Quay,NaN,Newry,BT35 6BP,United Kingdom,44 28 3025 2242,44 28 3025 2060,http://www.firstderivatives.com,Information Technology Services,Technology,First Derivatives plc provides software and co...,2400.0,[],86400,FDP
94,Worcester Road,NaN,Kidderminster,DY10 1JR,United Kingdom,44 15 6274 9300,44 15 6274 9649,http://www.victoriaplc.com,"Furnishings, Fixtures & Appliances",Consumer Cyclical,"Victoria PLC designs, manufactures, and distri...",3400.0,[],86400,VCP


In [11]:
symbols2 = dfg3['symbol']
symbols2

0      PAF
2      IOM
3     YNGN
5      ASC
6     MIDW
      ... 
91    ABDP
92     IQE
93     FDP
94     VCP
96    IDEA
Name: symbol, Length: 82, dtype: object

In [12]:
def get_groups(df, column_name):
    '''list symbols that belong to group'''
    df_tmp = df[[column_name, 'symbol']].groupby(column_name).agg(list)
    return {name: symbols[0] for name, symbols in df_tmp.iterrows()}

In [13]:
dict_sectors = get_groups(dfg3, 'sector')
dict_hierarchy = {'total': list(dict_sectors.keys())}
dict_hierarchy.update(dict_sectors)
dict_hierarchy

{'total': ['Basic Materials',
  'Communication Services',
  'Consumer Cyclical',
  'Consumer Defensive',
  'Energy',
  'Financial Services',
  'Healthcare',
  'Industrials',
  'Real Estate',
  'Technology'],
 'Basic Materials': ['PAF', 'HGM', 'ATYM', 'BMN', 'SCPA', 'CAML'],
 'Communication Services': ['KWS', 'TRMR', 'FDEV', 'GAMA', 'NFC', 'YOU'],
 'Consumer Cyclical': ['YNGN',
  'ASC',
  'CVSG',
  'IGR',
  'BOO',
  'TUNE',
  'APGN',
  'YNGA',
  'WJG',
  'ABDP',
  'VCP'],
 'Consumer Defensive': ['LTG', 'FEVR', 'HOTC', 'MPE', 'CAM', 'NICL'],
 'Energy': ['HUR', 'SQZ'],
 'Financial Services': ['IPX',
  'MAB1',
  'NUM',
  'BRK',
  'POLR',
  'RQIH',
  'PMI',
  'GROW',
  'MTW'],
 'Healthcare': ['ABC',
  'BVXP',
  'HCM',
  'CREO',
  'BMK',
  'SLN',
  'CTH',
  'HZD',
  'CRW',
  'AMS',
  'APH',
  'EMIS',
  'CLIN'],
 'Industrials': ['TFW',
  'DTG',
  'DATA',
  'JSG',
  'GTLY',
  'MRL',
  'JHD',
  'RWS',
  'RST',
  'CHRT',
  'RNWH',
  'SMS',
  'CWR'],
 'Real Estate': ['PURP'],
 'Technology': ['IOM

# Reconcile Tickers
Keep only symbols in both tables

In [14]:
to_drop = set(symbols) - set(symbols2)
to_drop

{'BREE', 'BUR', 'SIR'}

In [15]:
df5 = df4.drop(columns=to_drop)
df5['total'] = df5.sum(axis=1)
for sector, group in dict_hierarchy.items():
    if sector == 'total':
        continue
    df5[sector] = df5[group].sum(axis=1)
df5.index = pd.to_datetime(df5.index, format='%Y-%m-%d')
df5 = df5.resample('D').interpolate()
df5

symbol,ABC,ABDP,AMS,APGN,APH,ASC,ATYM,BMK,BMN,BOO,...,Basic Materials,Communication Services,Consumer Cyclical,Consumer Defensive,Energy,Financial Services,Healthcare,Industrials,Real Estate,Technology
Date,,,,,,,,,,,,,,,,,,,,,
2017-01-03,747.474426,544.871216,217.710587,366.378662,42.915035,5081.000000,123.000000,81.362900,1.925000,1.370000,...,761.507632,1949.483078,10153.199986,13643.282246,64.120498,3879.133610,6075.901547,4324.632859,143.000000,5706.782598
2017-01-04,735.387268,537.453857,216.236252,373.805267,44.329815,5038.000000,125.000000,81.362900,1.775000,1.340000,...,766.584904,1945.147537,10125.196231,13620.701314,64.716552,3884.574871,6081.325405,4326.076797,140.250000,5766.634346
2017-01-05,740.222107,540.915039,214.270432,373.805267,44.565609,5124.000000,137.500000,82.860397,1.825000,1.385000,...,789.390330,1954.457672,10224.189138,13550.622109,67.594769,3898.844807,6108.201123,4363.271866,145.000000,5789.986723
2017-01-06,741.189087,530.037415,211.813232,388.658447,44.683510,5256.000000,142.000000,82.860397,1.975000,1.407500,...,787.972787,1971.875367,10365.445586,13474.522774,67.972984,3966.840354,6065.965843,4352.342012,152.750000,5828.336391
2017-01-07,743.123067,530.861674,211.649414,391.133972,44.801407,5275.666667,145.166667,83.193166,1.991667,1.416667,...,792.355581,1975.982015,10394.153488,13423.010422,68.052029,3982.942621,6091.928799,4347.543339,152.500000,5830.943375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-03,1300.000000,1675.000000,194.399994,330.000000,71.300003,3465.000000,208.000000,41.000000,11.875000,270.299988,...,790.774995,6849.000000,10096.999985,9676.509996,122.449998,4972.500000,10504.699997,6950.500000,51.000000,8734.300003
2020-08-04,1259.000000,1675.000000,193.800003,330.000000,72.599998,3523.000000,209.000000,40.500000,13.000000,265.000000,...,800.799997,6845.000000,10172.699997,9612.970003,125.085000,4970.400002,10511.400001,7034.900002,54.099998,8810.899994
2020-08-05,1254.000000,1720.000000,198.399994,330.000000,69.099998,3899.000000,208.500000,41.000000,13.500000,275.000000,...,812.699991,7055.000000,10692.899994,9597.179995,127.950003,4973.400002,10635.999992,7089.500000,54.799999,8969.150001


# (Optional) Review Hierarchy Tree
Look at hierarchy tree

In [16]:
ht = HierarchyTree.from_nodes(nodes=dict_hierarchy, df=df5)
print(ht)

- total
   |- Basic Materials
   |  |- PAF
   |  |- HGM
   |  |- ATYM
   |  |- BMN
   |  |- SCPA
   |  - CAML
   |- Communication Services
   |  |- KWS
   |  |- TRMR
   |  |- FDEV
   |  |- GAMA
   |  |- NFC
   |  - YOU
   |- Consumer Cyclical
   |  |- YNGN
   |  |- ASC
   |  |- CVSG
   |  |- IGR
   |  |- BOO
   |  |- TUNE
   |  |- APGN
   |  |- YNGA
   |  |- WJG
   |  |- ABDP
   |  - VCP
   |- Consumer Defensive
   |  |- LTG
   |  |- FEVR
   |  |- HOTC
   |  |- MPE
   |  |- CAM
   |  - NICL
   |- Energy
   |  |- HUR
   |  - SQZ
   |- Financial Services
   |  |- IPX
   |  |- MAB1
   |  |- NUM
   |  |- BRK
   |  |- POLR
   |  |- RQIH
   |  |- PMI
   |  |- GROW
   |  - MTW
   |- Healthcare
   |  |- ABC
   |  |- BVXP
   |  |- HCM
   |  |- CREO
   |  |- BMK
   |  |- SLN
   |  |- CTH
   |  |- HZD
   |  |- CRW
   |  |- AMS
   |  |- APH
   |  |- EMIS
   |  - CLIN
   |- Industrials
   |  |- TFW
   |  |- DTG
   |  |- DATA
   |  |- JSG
   |  |- GTLY
   |  |- MRL
   |  |- JHD
   |  |- RWS
   |  |-

# Try HTS Regressor
Try hierarchical forecast

In [17]:
%%time
clf = HTSRegressor(model='prophet', revision_method='OLS')
model = clf.fit(df=df5, nodes=dict_hierarchy)

Fitting models:   0%|          | 0/19 [00:00<?, ?it/s]INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling dai

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
Fitting models:  68%|██████▊   | 13/19 [00:18<00:08,  1.43s/it]INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
Fitting models:  74%|███████▎  | 14/19 [00:19<00:06,  1.38s/it]INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet wi

CPU times: user 500 ms, sys: 422 ms, total: 922 ms
Wall time: 25.5 s


In [18]:
preds = model.predict(steps_ahead=60)
preds

Fitting models: 100%|██████████| 19/19 [00:58<00:00,  3.08s/it]


,total,Basic Materials,Communication Services,Consumer Cyclical,Consumer Defensive,Energy,Financial Services,Healthcare,Industrials,Real Estate,...,WAND,TCM,DOTD,TRCS,GHH,ITM,IMO,IQE,FDP,IDEA
2017-01-03,7057.248959,1758.546199,-722.437618,-1629.836033,-3940.596571,-4430.835594,-5723.283889,-5498.349811,-5998.126214,-3265.336962,...,2370.059034,2112.714797,2254.950918,2288.040208,2338.566639,2132.074632,2530.337920,3081.623873,2100.770383,2220.040200
2017-01-04,7100.964091,1767.115762,-721.721484,-1645.413793,-3968.071296,-4458.535615,-5760.736690,-5533.135175,-6035.637808,-3284.792263,...,2384.133084,2125.539692,2282.661272,2308.974367,2351.711167,2144.724974,2543.001677,3101.064833,2113.142958,2234.443208
2017-01-05,7134.171561,1772.860538,-726.540320,-1650.227595,-3988.484096,-4476.632176,-5788.673615,-5557.864489,-6062.180495,-3298.223547,...,2393.283495,2135.099202,2306.689172,2324.572850,2361.413499,2154.160274,2553.155051,3115.168264,2121.802061,2245.817679
2017-01-06,7185.703628,1790.832747,-729.357899,-1666.035808,-4021.898633,-4508.514037,-5831.056517,-5602.269997,-6105.489162,-3318.200029,...,2407.916752,2150.025276,2335.575956,2347.141032,2377.296639,2168.672075,2569.222862,3140.769804,2136.099831,2262.597066
2017-01-07,7214.365696,1796.642159,-718.058754,-1676.918100,-4040.931343,-4525.851327,-5855.119407,-5629.489328,-6129.374966,-3331.443626,...,2414.890975,2158.332493,2352.759396,2362.413967,2385.813598,2176.613206,2578.109851,3155.345036,2144.113409,2271.874988
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-10-02,9469.144151,848.435182,2411.547264,-1644.604626,-6746.805825,-6173.522394,-7851.096161,-6114.770450,-8038.811347,-3628.917862,...,3216.895671,2768.664519,3757.848126,3084.437739,2637.179204,2637.928951,3155.312784,3612.992840,2887.541544,2821.838039
2020-10-03,9448.534218,846.197816,2402.894007,-1628.667558,-6738.593321,-6161.104816,-7833.520873,-6102.528244,-8018.458733,-3619.927321,...,3211.014561,2764.302461,3745.921814,3075.796637,2631.910144,2633.138417,3149.729790,3607.376343,2883.564263,2816.513210
2020-10-04,9433.016451,846.200388,2389.104571,-1613.658106,-6733.289356,-6151.765448,-7820.037252,-6093.440223,-8002.728255,-3612.710983,...,3207.078064,2761.413342,3736.770644,3068.203323,2628.400817,2630.004529,3145.374161,3603.378992,2880.986851,2813.042138
2020-10-05,9418.266478,846.288499,2374.096493,-1598.868270,-6728.277182,-6142.894110,-7807.306378,-6085.141618,-7987.761430,-3605.950160,...,3203.570651,2758.770117,3728.201916,3060.648992,2625.183021,2627.135070,3141.111894,3599.866917,2878.651077,2809.823372


In [25]:
%%time
clf = HTSRegressor(model='holt_winters', revision_method='OLS')
model = clf.fit(df=df5, nodes=dict_hierarchy)




Fitting models:   0%|          | 0/19 [00:00<?, ?it/s]


Fitting models:  21%|██        | 4/19 [00:00<00:00, 39.25it/s]


Fitting models: 100%|██████████| 19/19 [00:00<00:00, 80.94it/s]


CPU times: user 203 ms, sys: 391 ms, total: 594 ms
Wall time: 698 ms


In [27]:
%%time
preds = model.predict(steps_ahead=60)
preds





Fitting models:   0%|          | 0/19 [00:00<?, ?it/s]



Fitting models:  11%|█         | 2/19 [00:00<00:00, 19.01it/s]



Fitting models: 100%|██████████| 19/19 [00:00<00:00, 63.64it/s]


CPU times: user 125 ms, sys: 438 ms, total: 562 ms
Wall time: 801 ms


,total,Basic Materials,Communication Services,Consumer Cyclical,Consumer Defensive,Energy,Financial Services,Healthcare,Industrials,Real Estate,...,WAND,TCM,DOTD,TRCS,GHH,ITM,IMO,IQE,FDP,IDEA
2017-01-03,2655.432112,7005.184681,2258.961953,-1128.766955,190.281112,2272.798607,-1760.613120,-1582.834868,-1740.361932,-1092.487657,...,-302.734799,-227.472468,-460.484799,1637.202457,-431.748265,-440.193886,4.376560,512.380802,-476.688301,-341.234799
2017-01-04,2655.627017,7005.017620,2260.224102,-1128.945617,191.045757,2272.579325,-1760.816325,-1582.998355,-1740.543191,-1092.585109,...,-302.775632,-227.513303,-460.525632,1637.161624,-431.789098,-440.234719,4.335727,512.339969,-476.726872,-341.275632
2017-01-05,2655.876080,7008.255942,2244.379832,-1132.342181,175.286573,2310.022865,-1763.442840,-1581.476610,-1741.984029,-1095.865283,...,-305.686715,-230.214935,-462.936715,1646.926322,-435.700181,-444.145802,-4.527169,518.193168,-480.731289,-345.186715
2017-01-06,2667.818915,7040.258374,2251.921194,-1137.556107,165.924283,2318.516425,-1772.549965,-1584.873368,-1739.022924,-1099.558527,...,-329.380158,-230.446928,-464.630158,1620.311248,-439.878312,-446.339245,-6.720612,527.717010,-482.759950,-337.880158
2017-01-07,2675.937563,7053.657576,2252.648214,-1133.435453,158.866335,2338.676973,-1773.701225,-1597.736116,-1743.403632,-1099.789661,...,-332.190622,-233.940381,-466.690622,1600.893607,-440.207918,-449.149709,-9.531076,581.540091,-485.717588,-344.190622
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-10-02,3626.212564,9704.310666,3540.889804,-2164.589098,338.813049,3897.573540,-2686.884822,-2149.049139,-2555.269982,-1713.781281,...,-215.235122,-654.190817,-729.835120,2158.764878,-615.235122,-663.235122,-216.235122,228.764878,-528.703947,-423.235122
2020-10-03,3626.212564,9704.310666,3540.889804,-2164.589098,338.813049,3897.573540,-2686.884822,-2149.049139,-2555.269982,-1713.781281,...,-215.235122,-654.190817,-729.835120,2158.764878,-615.235122,-663.235122,-216.235122,228.764878,-528.703947,-423.235122
2020-10-04,3626.212564,9704.310666,3540.889804,-2164.589098,338.813049,3897.573540,-2686.884822,-2149.049139,-2555.269982,-1713.781281,...,-215.235122,-654.190817,-729.835120,2158.764878,-615.235122,-663.235122,-216.235122,228.764878,-528.703947,-423.235122
2020-10-05,3626.212564,9704.310666,3540.889804,-2164.589098,338.813049,3897.573540,-2686.884822,-2149.049139,-2555.269982,-1713.781281,...,-215.235122,-654.190817,-729.835120,2158.764878,-615.235122,-663.235122,-216.235122,228.764878,-528.703947,-423.235122
